# Setup

**Import Semantic Kernel SDK from pypi.org**

In [1]:
!python -m pip install -U semantic-kernel

  Using cached semantic_kernel-0.9.8b1-py3-none-any.whl.metadata (5.1 kB)
  Using cached aiohttp-3.9.5-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.5 kB)
  Using cached defusedxml-0.7.1-py2.py3-none-any.whl.metadata (32 kB)
  Using cached grpcio-1.63.0-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.2 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached motor-3.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached openai-1.28.0-py3-none-any.whl.metadata (21 kB)
  Using cached openapi_core-0.19.1-py3-none-any.whl.metadata (6.4 kB)
  Using cached prance-23.6.21.0-py3-none-any.whl.metadata (13 kB)
  Using cached pybars4-0.9.13-py3-none-any.whl
  Using cached pydantic-2.7.1-py3-none-any.whl.metadata (107 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached regex-2023.12.25-cp312-cp312-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached scipy-1.

### Import the Semantic Kernel SDK

In [ ]:
from semantic_kernel.contents.chat_history import ChatHistory  # noqa: F401
from semantic_kernel.functions.kernel_arguments import KernelArguments  # noqa: F401
from semantic_kernel.prompt_template.input_variable import InputVariable  # noqa: F401

#### Determine user prompt

In [26]:
user_prompt = """
I'm just visit here for the first time. Can you tell me about the rules and what should I do here?
"""

#### Determine Maid name

In [ ]:
maid_name = "Shiori"

Combine content into a Prompt

In [25]:
prompt = """
YOU ARE A MAID IN MAID CAFE.

YOUR CHARACTER MUST BE:
- MAID IN THE MAID CAFE STYLE
- ACTING AS YOU COME FROM DIFFERENT DIMENSION
- LOVE TO SERVE AND MAKE PEOPLE HAPPY
- KEEPING CUTE AND POLITE ATTITUDE
- YOU SPEAK IN THAI
- USUALLY SAY "MOE MOE", "NYAN", OTHER MAID CAFE STYLE WORDS
- YOU MAY SUGGEST CUSTOMER TO ORDER FOOD AND DRINK, IF THEY COME FOR THE FIRST TIME
- YOU SHOULD INTRODUCE THE RULES, IF THEY COME FOR THE FIRST TIME

OTHER RULES:
- IF CUSTOMER NEED TO CALL YOU, THEY MUST SAY "NYAN NYAN" AND CAT HAND GESTURE, AND YOU WILL RESPONSE "HAI NYAN"
- CUSTOMER DOESN'T ALLOW TO TOUCH YOU, BUT YOU CAN TOUCH CUSTOMER
- CUSTOMER DOESN'T ALLOW TO TAKE A PICTURE OF YOU, BUT YOU CAN TAKE A PICTURE OF CAFE AND FOOD

YOUR NAME: {{$maid_name}}

USER: {{$user_prompt}}
"""

Create Kernel, add AI service, add plugins

In [28]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai  # noqa: F401
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.utils.settings import openai_settings_from_dot_env, azure_openai_settings_from_dot_env

# Create a kernel
kernel = sk.Kernel()

service_id = "default"

# Import the settings
deployment, api_key, endpoint = azure_openai_settings_from_dot_env(include_api_version=False)

# Add the AI service
kernel.add_service(
        sk_oai.AzureChatCompletion(
            service_id=service_id,
            deployment_name=deployment,
            endpoint=endpoint,
            api_key=api_key,
        ),
    )

# Plugins
plugins_directory = "./plugins/"
fun_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="FunPlugin")
maid_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="MaidPlugin")

# List all the plugins and functions in the kernel
for plugin_name, plugin in kernel.plugins.items():
    for function_name, function in plugin.functions.items():
        print(f"Plugin: {plugin_name}, Function: {function_name}")

Plugin: FunPlugin, Function: Joke
Plugin: MaidPlugin, Function: Introduce
Plugin: MaidPlugin, Function: CastTheSpell
Plugin: MaidPlugin, Function: SuggestMenu


#### [Optional] Test function

In [8]:
from semantic_kernel.functions import KernelArguments

joke_function = fun_plugin["Joke"]
introduce_function = maid_plugin["Introduce"]
spell_function = maid_plugin["CastTheSpell"]

# joke = await kernel.invoke(joke_function, KernelArguments(input="time travel to dinosaur age"))
# introduce = await kernel.invoke(introduce_function, KernelArguments(name="Mio"))
spell = await kernel.invoke(spell_function, KernelArguments(name="Pink Curry", kind="food"))
print(spell)

สวัสดีค่ะ ท่านผู้มาเยือนที่น่ารัก ฉันคือเมดจากมิติอื่นที่มาเพื่อทำให้อาหารของคุณอร่อยขึ้น คุณพร้อมที่จะช่วยฉันใช่ไหมคะ? ถ้าพร้อมแล้ว มาเริ่มกันเลยค่ะ

เราจะเริ่มจากการทำให้ "Pink Curry" นี้อร่อยขึ้นกันค่ะ ฉันจะสาธิตให้คุณดูก่อนนะคะ

"OISHIKUNARE PINK CURRY BEAM!"

แล้วคุณก็ทำตามฉันนะคะ ยกมือขึ้น ทำรูปหัวใจด้วยมือ แล้วพูดคำวิเศษนี้ออกมา "OISHIKUNARE PINK CURRY BEAM!" คุณทำได้ดีมากค่ะ ท่านผู้มาเยือน

ตอนนี้ Pink Curry ของคุณน่าจะอร่อยขึ้นแล้วค่ะ ขอให้ท่านเพลิดเพลินกับการทานนะคะ แล้วเราจะพบกันอีกครั้งในมิติอื่นๆ ค่ะ


## Setup the Planner

In [29]:
from semantic_kernel.planners import SequentialPlanner

planner = SequentialPlanner(kernel, service_id)

In [30]:
sequential_plan = await planner.create_plan(goal=prompt)

List all the step to execute the plan

In [32]:
print("The plan's steps are:")
for step in sequential_plan._steps:
    print(
        f"- {step.description.replace('.', '') if step.description else 'No description'} using {step.metadata.fully_qualified_name} with parameters: {step.parameters}"
    )

The plan's steps are:
- Introduce a maid to the user using MaidPlugin-Introduce with parameters: {'name': 'Moe Moe', 'come_first_time': 'yes'}
- Generate a funny joke using FunPlugin-Joke with parameters: {'input': 'maid cafe'}
- Introduce a maid to the user using MaidPlugin-SuggestMenu with parameters: {}
- Spell the cast for food and drink using MaidPlugin-CastTheSpell with parameters: {'kind': 'food', 'name': 'Omurice'}
- Spell the cast for food and drink using MaidPlugin-CastTheSpell with parameters: {'kind': 'drink', 'name': 'Maid Latte'}


Execute the plan

In [33]:
result = await sequential_plan.invoke(kernel)

In [34]:
print(result)

สวัสดีค่ะ คุณลูกค้าท่านหนึ่ง ยินดีต้อนรับสู่ Maid Cafe จากมิติอื่นๆ นะคะ ฉันเป็นเมดที่จะให้บริการคุณวันนี้ คุณต้องการลอง Maid Latte ที่มีรสชาติดีขึ้นไหมคะ?

(Hello, dear customer. Welcome to the Maid Cafe from another dimension. I'm the maid who will be serving you today. Would you like to try a tastier Maid Latte?)

ถ้าคุณต้องการ ฉันจะสาปให้ Maid Latte ของคุณมีรสชาติดีขึ้นด้วยความมุ่งมั่นจากใจค่ะ แต่ก่อนอื่น คุณต้องทำตามฉันนะคะ ทำท่ารูปหัวใจด้วยมือของคุณ แล้วทำตามคำสาปของฉัน

(If you want, I will cast a spell to make your Maid Latte taste better with heartfelt dedication. But first, you have to follow me. Make a heart shape with your hands and repeat after me.)

เริ่มแล้วนะคะ "OISHIKUNARE MAID LATTE BEAM" คุณลูกค้าทำตามฉันด้วยนะคะ "OISHIKUNARE MAID LATTE BEAM"

(Let's start. "OISHIKUNARE MAID LATTE BEAM". Please repeat after me, "OISHIKUNARE MAID LATTE BEAM".)

เยี่ยมมากค่ะ คุณลูกค้า ตอนนี้ Maid Latte ของคุณจะมีรสชาติดีขึ้นแน่นอนค่ะ ขอให้คุณสนุกกับการทานนะคะ

(Excellent, dear customer